In [39]:
#-------------SVMスクラッチ-----------

class ScratchSVMClassifier:
    
    """
    SVM分類器のスクラッチ実装
    Parameters
    ----------
    num_iter : int
      イテレーション数
    alfa : float
      学習率
    kernel : str
      カーネルの種類。線形カーネル（linear）か多項式カーネル（polly）
    threshold : float
      サポートベクターを選ぶための閾値
    verbose : bool
      学習過程を出力する場合はTrue
    Attributes
    ----------
    self.n_support_vectors : int
      サポートベクターの数
    self.index_support_vectors : 次の形のndarray, shape (n_support_vectors,)
      サポートベクターのインデックス
    self.X_sv :  次の形のndarray, shape(n_support_vectors, n_features)
      サポートベクターの特徴量
    self.lam_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターの未定乗数
    self.S :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターの配列
    self.y_hat :  次の形のndarray, shape(n_support_vectors, 1)

    """
    def __init__(self, num_iter, alfa, kernel='linear', threshold=1e-5, verbose=False):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.alfa = alfa
        self.kernel = kernel
        self.threshold = threshold
        self.verbose = verbose
        
        # 目的関数の値を記録する配列を用意
        self.s = np.array([])                     #サポートベクターの配列
        self.index_support_vectors = np.array([]) #サポートベクターのラベル(yの値)
        self.n_support_vectors = np.array([])     #サポートベクターの要素数
        self.lam_sv = np.array([])                #ラムダ(サポートベクターの未定乗数)
        self.X_sv = np.array([])                  #サポートベクター


    #ラグランジュの未定乗数法による最急降下    
    def fit(self, X, y, X_val=None, y_val=None):
        """
        SVM分類器を学習する。検証データが入力された場合はそれに対する精度もイテレーションごとに計算する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """

        #①テストデータでラグランジュ乗数λを計算
        self.lam_sv = Lagrange(self,X,y)

        #②テストデータでサポートベクターの決定
        self.s, self.index_support_vectors = suport_victor()


    #ラグランジュ乗数λの計算部分
    def Lagrange(self,X,y):

        #self.lam_svをXのデータ数分のゼロ配列と定義
        self.lam_sv = np.zeros(X.shape[0])

        sum_siguma = 0 #初期値の定義

        #データの数
        n =  X.shape[0]

        #ダイバーのラグランジュの未定乗数法の公式
        for i in range( n ):

            for j in range( n ):
                siguma = self.lam_sv[i] * y[i] * y[j] * X[i,:].T @ X[j,:]
                sum_siguma += siguma

                self.lam_sv[i] = self.lam_sv[i] + self.alfa * (1 - sum_siguma)

        return self.lam_sv

    #サポートベクターの決定
    def suport_victor(self):
        """
        Parameters
        ----------
        self.lam_sv: ラグランジュ乗数λ（ndarray）
        be-ta : ハイパーパラメータ（float）

        Attributes
        ----------
        self.s : サポートベクター（ndarray）
        self.index_support_vectors : サポートベクターの要素に対応したｙ（ndarray）
        """

        #サポートベクターの要素番号の配列 (Xの中のハイパーパラメータより大きい要素No)
        s_num = np.where(self.lam_sv > 0.005 ) #(array([0, 1, 5], dtype=int64),)

        #s_numhaは配列とタイプのタプルで出力されるのでarrayのみ指定した。
        s_num = s_num[0]

        #空のサポートベクターの要素数を定義
        self.n_support_vectors = len(s_num)

        #サポートベクターの配列
        self.s = X[s_num]
        print("self.s:{}".format(self.s)) #------検証中--------

        #サポートベクターのインデックス
        self.index_support_vectors = y[s_num]
        print("self.index_support_vectors:{}".format(self.index_support_vectors)) #------検証中--------

        return self.s, self.index_support_vectors

    #予測値の推定
    def predict(self,X):

        for n in range(self.n_support_vectors):
            y_hat = self.lam_sv[n] * self.index_support_vectors[n] * X.T @ self.s[n,:]
        return y_hat


In [40]:
#-----------検証用データ-----------

import numpy as np
np.random.seed(seed=0)
n_samples = 10
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X = np.concatenate([f0, f1])
y = np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])

In [41]:
#-------------訓練データと検証データに分割-----------

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)


In [42]:
#-------------学習と推定-----------

#ScratchSVMClassifierをsample1でインスタンス化
sample1 = ScratchSVMClassifier(num_iter = 10, alfa = 0.1, kernel='linear', threshold=1e-5, verbose=False)

#sample1でfitメソッドを呼び出し
lam_sv = sample1.Lagrange(X_train,y_train)

print(lam_sv)

#サポートベクターの決定を呼び出し
sample1.suport_victor()

y_hat = sample1.predict(X_test)


[-9.71829031e-02 -5.66569783e-04 -1.26660615e-02  4.64724595e-03
 -1.75503636e-03  2.92068392e-04  6.46789974e-05]
self.s:[]
self.index_support_vectors:[]


UnboundLocalError: local variable 'y_hat' referenced before assignment

-------------エラーが出た項目-----------

●AttributeError: 'numpy.ndarray' object has no attribute 'lam_sv'
lam_svを空のndarrayとして定義せず使った
⇒self.lam_sv = np.array([])とすることで解決した。設計図に落とし込めていない

●TypeError: Lagrange() missing 1 required positional argument: 'y'
クラス内のメソッド
def Lagrange(self,X,y):
をクラスの外で
self.lam_sv = sample1.Lagrange(X,y)
と呼びだしたら引数ｙが足りないとエラー
⇒クラスの呼び出しの最後に（）を付けてない。クラスの呼び出しの（）の中身が__init__の（）と同期している事を忘れていた。

●TypeError: 'int' object is not iterable
for i in self.n_support_vectors: としたが上記エラーになる。self.n_support_vectorsはintで数字は3なのにできない.
⇒rangeを付けるのを忘れていた。また[0.1.5]の要素をforで取り出そうとしたがX[0.1.5]で取れることが分かり
ndarrayの操作の基礎ができてない事が分かった。

●メソッドを呼び出すときに
インスタンス名.メソッド名なのにdef メソッド名()としてしまった。